In [55]:
from nba_api.stats.static import teams
from nba_api.live.nba.endpoints import scoreboard
from utils import fetch_data, fetch_player_stats
from nba_api.stats.endpoints import teamdetails, teamyearbyyearstats, commonteamroster, teamgamelog, leaguegamefinder
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
import re

## Basics

https://github.com/swar/nba_api/blob/master/docs/examples/Basics.ipynb

## Teams

In [46]:
team_id = 1610612738 # celtics https://www.nba.com/stats/team/1610612738
previous_season = datetime.now().year - 1
# game_log = fetch_data(teamgamelog.TeamGameLog, team_id=team_id, season=previous_season)
game_log = teamgamelog.TeamGameLog(team_id=team_id, season=previous_season).get_data_frames()[0]
game_log

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612738,0022301186,"APR 14, 2024",BOS vs. WAS,W,64,18,0.780,240,51,...,0.667,10,38,48,29,10,15,14,13,132
1,1610612738,0022301173,"APR 12, 2024",BOS vs. CHA,W,63,18,0.778,240,54,...,0.692,16,37,53,34,12,1,6,20,131
2,1610612738,0022301167,"APR 11, 2024",BOS vs. NYK,L,62,18,0.775,240,40,...,0.813,10,26,36,25,7,12,12,16,109
3,1610612738,0022301148,"APR 09, 2024",BOS @ MIL,L,62,17,0.785,240,37,...,0.000,12,26,38,27,11,4,12,8,91
4,1610612738,0022301134,"APR 07, 2024",BOS vs. POR,W,62,16,0.795,240,49,...,0.769,5,35,40,34,11,8,12,12,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,1610612738,0022300136,"NOV 04, 2023",BOS @ BKN,W,5,0,1.000,240,43,...,0.852,10,40,50,22,4,6,11,17,124
78,1610612738,0022300118,"NOV 01, 2023",BOS vs. IND,W,4,0,1.000,240,54,...,0.964,11,46,57,27,5,2,11,19,155
79,1610612738,0022300103,"OCT 30, 2023",BOS @ WAS,W,3,0,1.000,240,51,...,0.714,15,36,51,31,11,6,17,21,126
80,1610612738,0022300080,"OCT 27, 2023",BOS vs. MIA,W,2,0,1.000,240,45,...,0.684,16,39,55,20,7,6,15,19,119


In [51]:
g = game_log.iloc[42]
g

Team_ID        1610612738
Game_ID        0022300563
GAME_DATE    JAN 15, 2024
MATCHUP         BOS @ TOR
WL                      W
W                      31
L                       9
W_PCT               0.775
MIN                   240
FGM                    33
FGA                    82
FG_PCT              0.402
FG3M                   16
FG3A                   39
FG3_PCT              0.41
FTM                    23
FTA                    25
FT_PCT               0.92
OREB                    5
DREB                   48
REB                    53
AST                    20
STL                     3
BLK                     7
TOV                    12
PF                     16
PTS                   105
Name: 42, dtype: object

In [52]:
matchup = g['MATCHUP'].replace(' vs. ', '-vs-').replace(' @ ', '-vs-').lower()
game_id = g['Game_ID']
print(f"https://www.nba.com/game/{matchup}-{game_id}")

https://www.nba.com/game/bos-vs-tor-0022300563


In [59]:
league_game_finder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2023-24')
games = league_game_finder.get_data_frames()[0]
games[games['GAME_ID'] == '0022300563']


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1466,22023,1610612761,TOR,Toronto Raptors,0022300563,2024-01-15,TOR vs. BOS,L,241,96,...,0.762,9,43,52,25,6,2,7,21,-9.0
1467,22023,1610612738,BOS,Boston Celtics,0022300563,2024-01-15,BOS @ TOR,W,240,105,...,0.920,5,48,53,20,3,7,12,16,9.0


## Live Data


https://github.com/swar/nba_api/blob/master/docs/examples/LiveData.ipynb

In [17]:
games = scoreboard.ScoreBoard()
# games.get_dict()
games.get_dict()['scoreboard']['games'][0]


{'gameId': '0012400046',
 'gameCode': '20241014/PHIATL',
 'gameStatus': 2,
 'gameStatusText': 'Q1 7:21',
 'period': 1,
 'gameClock': 'PT07M21.00S',
 'gameTimeUTC': '2024-10-14T23:30:00Z',
 'gameEt': '2024-10-14T19:30:00-04:00',
 'regulationPeriods': 4,
 'ifNecessary': False,
 'seriesGameNumber': '',
 'gameLabel': None,
 'gameSubLabel': None,
 'seriesText': '',
 'seriesConference': '',
 'poRoundDesc': '',
 'gameSubtype': '',
 'homeTeam': {'teamId': 1610612737,
  'teamName': 'Hawks',
  'teamCity': 'Atlanta',
  'teamTricode': 'ATL',
  'wins': 0,
  'losses': 0,
  'score': 11,
  'seed': None,
  'inBonus': '0',
  'timeoutsRemaining': 7,
  'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 11},
   {'period': 2, 'periodType': 'REGULAR', 'score': 0},
   {'period': 3, 'periodType': 'REGULAR', 'score': 0},
   {'period': 4, 'periodType': 'REGULAR', 'score': 0}]},
 'awayTeam': {'teamId': 1610612755,
  'teamName': '76ers',
  'teamCity': 'Philadelphia',
  'teamTricode': 'PHI',
  'wins': 0,
 

In [7]:
def get_today_games():
    today = datetime.now().date()
    date_str = today.strftime("%Y-%m-%d")
    
    board = scoreboard.ScoreBoard()
    games_today = board.games.get_dict()
    
    games = []
    for game in games_today:
        game_info = {
            'Game ID': game['gameId'],
            'Date': date_str,
            'Home Team': game['homeTeam']['teamName'],
            'Away Team': game['awayTeam']['teamName'],
            'Home Odds': 2.0,  # Placeholder odds
            'Away Odds': 2.0,  # Placeholder odds
        }
        games.append(game_info)
    
    return pd.DataFrame(games)

In [8]:
get_today_games()

,Game ID,Date,Home Team,Away Team,Home Odds,Away Odds
0,0012400046,2024-10-14,Hawks,76ers,2.0,2.0
1,0012400045,2024-10-14,Pacers,Grizzlies,2.0,2.0
2,0012400047,2024-10-14,Nets,Wizards,2.0,2.0
3,0012400048,2024-10-14,Bucks,Bulls,2.0,2.0
4,0012400049,2024-10-14,Clippers,Mavericks,2.0,2.0


In [9]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

ScoreBoardDate: 2024-10-14
0012400046: 76ers vs. Hawks @ 2024-10-14 19:30:00-04:00
0012400045: Grizzlies vs. Pacers @ 2024-10-14 19:00:00-04:00
0012400047: Wizards vs. Nets @ 2024-10-14 19:30:00-04:00
0012400048: Bulls vs. Bucks @ 2024-10-14 20:00:00-04:00
0012400049: Mavericks vs. Clippers @ 2024-10-14 22:30:00-04:00


In [27]:
# Getting Box Scores. 
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0012400045') 
# Note: home_team & away_team have the identicial data structure.
away_team = box.away_team.get_dict()['teamName']
players = box.away_team.get_dict()['players']
f = "[{name}](https://www.nba.com/stats/player/{player_id}) ({team}): {points} PTS"
for player in players:
    print(f.format(name=player['name'],player_id=player['personId'],team=away_team,points=player['statistics']['points']))

[Jake LaRavia](https://www.nba.com/stats/player/1631222) (Grizzlies): 3 PTS
[Santi Aldama](https://www.nba.com/stats/player/1630583) (Grizzlies): 3 PTS
[Brandon Clarke](https://www.nba.com/stats/player/1629634) (Grizzlies): 2 PTS
[Desmond Bane](https://www.nba.com/stats/player/1630217) (Grizzlies): 11 PTS
[Scotty Pippen Jr.](https://www.nba.com/stats/player/1630590) (Grizzlies): 3 PTS
[Luke Kennard](https://www.nba.com/stats/player/1628379) (Grizzlies): 3 PTS
[Jaylen Wells](https://www.nba.com/stats/player/1642377) (Grizzlies): 2 PTS
[Zach Edey](https://www.nba.com/stats/player/1641744) (Grizzlies): 6 PTS
[Yuki Kawamura](https://www.nba.com/stats/player/1642530) (Grizzlies): 3 PTS
[John Konchar](https://www.nba.com/stats/player/1629723) (Grizzlies): 2 PTS
[Armando Bacot](https://www.nba.com/stats/player/1630654) (Grizzlies): 0 PTS
[Jay Huff](https://www.nba.com/stats/player/1630643) (Grizzlies): 0 PTS
[GG Jackson](https://www.nba.com/stats/player/1641713) (Grizzlies): 0 PTS
[Jaren Jack

## Finding Games

https://github.com/swar/nba_api/blob/master/docs/examples/Finding%20Games.ipynb

In [38]:
from nba_api.stats.endpoints import leaguegamefinder
league_game_finder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2024-25')
games = league_game_finder.get_data_frames()[0]
games


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,12024,1610612737,ATL,Atlanta Hawks,0012400046,2024-10-14,ATL vs. PHI,None,24,11,...,NaN,0,7,7,4,3,2,2,2,5.0
1,12024,1610612763,MEM,Memphis Grizzlies,0012400045,2024-10-14,MEM @ IND,None,119,47,...,0.750,6,17,23,14,5,0,4,7,16.8
2,12024,1610612755,PHI,Philadelphia 76ers,0012400046,2024-10-14,PHI @ ATL,None,0,6,...,NaN,1,6,7,1,0,0,4,3,-5.0
3,12024,1610612754,IND,Indiana Pacers,0012400045,2024-10-14,IND vs. MEM,None,120,34,...,0.750,2,18,20,8,2,3,8,4,-21.0
4,12024,1610612751,BKN,Brooklyn Nets,0012400047,2024-10-14,BKN vs. WAS,None,24,12,...,NaN,2,6,8,4,0,2,1,3,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,12024,1610612762,UTA,Utah Jazz,0012400002,2024-10-04,UTA vs. NZB,W,238,116,...,0.727,9,35,44,27,11,5,21,19,29.0
86,12024,1610612743,DEN,Denver Nuggets,0012400001,2024-10-04,BOS @ DEN,L,239,103,...,0.654,12,37,49,29,10,2,23,24,-4.0
87,12024,1610612747,LAL,Los Angeles Lakers,0012400003,2024-10-04,LAL vs. MIN,L,241,107,...,0.913,4,34,38,28,9,12,20,20,-17.0
88,12024,15020,NZB,New Zealand Breakers,0012400002,2024-10-04,NZB @ UTA,L,239,87,...,0.615,13,22,35,23,11,5,16,20,-29.0
